In [84]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
%matplotlib inline

In [2]:
df_US = pd.read_csv('USvideos.csv')
df_CA = pd.read_csv('CAvideos.csv')

In [3]:
df_US.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [4]:
df_CA.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


We see that our dataset contains many columns: some to identify the video and channel that published the video, some to keep track of viewer statistics, and some to describe the kind of video it is. The dataset also contains a 'thumbnail_link' column which contains a link to a page that contains the thumbnail for the video. Although it would be an interesting assignment to examine the picture and use it to understand how the choice of thumbnails can affect the likelihood of trending of a video, I think that might be too complicated an endeavor for me to currently embark on and so will remove it from the dataset.

In [5]:
cats_US = pd.read_json('UScategoryids.json')
cats_CA = pd.read_json('CAcategoryids.json')

In [6]:
cats_US.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


In [7]:
cats_CA.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
1,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
2,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
3,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."
4,youtube#videoCategoryListResponse,"""ld9biNPKjAjgjV7EZ4EKeEGrhao/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""ld..."


We see that cats['items'] appears to be a dictionary with a lot of information.

In [8]:
cats_US['items'][0]

{'kind': 'youtube#videoCategory',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
 'id': '1',
 'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title': 'Film & Animation',
  'assignable': True}}

Our assumption was correct! cats['items'] is a dictionary that contains various useful bits of information, including the category id and the category title, which, interestingly enough, is located in 'snippet' (which is a dictionary within a dictionary within a json).

In [9]:
categories_US = {int(category['id']): category['snippet']['title'] for category in cats_US['items']}
categories_CA = {int(category['id']): category['snippet']['title'] for category in cats_CA['items']}

In [10]:
#Converting from objects to datetime64

df_US.trending_date = pd.to_datetime(df_US.trending_date, format='%y.%d.%m', errors='coerce')
df_US.publish_time = pd.to_datetime(df_US.publish_time, format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')

df_CA.trending_date = pd.to_datetime(df_CA.trending_date, format='%y.%d.%m', errors='coerce')
df_CA.publish_time = pd.to_datetime(df_CA.publish_time, format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce')

In [11]:
#Creating a 'publish_date' column with just the date so that we can compare trend dates and publish dates

df_US['publish_date'] = df_US['publish_time'].dt.date
df_US['publish_date'] = pd.to_datetime(df_US['publish_date'], format='%Y-%m-%d', errors='coerce')

df_CA['publish_date'] = df_CA['publish_time'].dt.date
df_CA['publish_date'] = pd.to_datetime(df_CA['publish_date'], format='%Y-%m-%d', errors='coerce')

In [12]:
#Mapping values in a dictionary to its corresponding keys in the dataframe

df_US['category_name'] = df_US['category_id'].map(categories_US)
df_CA['category_name'] = df_CA['category_id'].map(categories_CA)

In [13]:
len(df_CA.category_id.unique())

17

In [14]:
len(df_US.category_id.unique())

16

In [15]:
df_CA.category_name.unique()

array(['Music', 'Comedy', 'Entertainment', 'News & Politics',
       'People & Blogs', 'Howto & Style', 'Film & Animation',
       'Science & Technology', 'Gaming', 'Sports', nan, 'Pets & Animals',
       'Travel & Events', 'Autos & Vehicles', 'Education', 'Shows',
       'Movies'], dtype=object)

In [16]:
df_CA[df_CA.category_name.isnull()]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_date,category_name
43,0gLWN3uq74o,2017-11-14,Ex-intelligence chiefs fire back at Trump crit...,CNN,29,2017-11-12 15:36:51,"James Clapper|""John Brennan""|""President Trump""...",286231,2530,1127,3239,https://i.ytimg.com/vi/0gLWN3uq74o/default.jpg,False,False,False,Former Director of National Intelligence James...,2017-11-12,NaN
228,HnCU20Cu0fs,2017-11-15,Dashcam Norway - Semi-truck narrowly missing kids,FBas,29,2017-11-15 07:37:18,"dashcam|""accident""|""traffic""|""safety""|""volvo""|...",2253298,4632,104,932,https://i.ytimg.com/vi/HnCU20Cu0fs/default.jpg,False,False,False,Copyright Transferd AS - a part of the Firda B...,2017-11-15,NaN
359,0gLWN3uq74o,2017-11-15,Ex-intelligence chiefs fire back at Trump crit...,CNN,29,2017-11-12 15:36:51,"James Clapper|""John Brennan""|""President Trump""...",311936,2627,1196,3621,https://i.ytimg.com/vi/0gLWN3uq74o/default.jpg,False,False,False,Former Director of National Intelligence James...,2017-11-12,NaN
385,ClwBTkLiivk,2017-11-15,Week 4 Challenge: Check out “My Email Leads”,CREA | ACI,29,2017-11-13 13:46:37,CREA,1898,0,0,0,https://i.ytimg.com/vi/ClwBTkLiivk/default.jpg,True,False,False,www.realtor.ca/2mins,2017-11-13,NaN
489,HnCU20Cu0fs,2017-11-16,ORIGINAL: Dashcam Norway - Semi truck narrowly...,Transferd AS,29,2017-11-15 07:37:18,"dashcam|""accident""|""traffic""|""safety""|""volvo""|...",2510664,0,0,1641,https://i.ytimg.com/vi/HnCU20Cu0fs/default.jpg,False,True,False,Copyright Transferd AS - a part of the Firda B...,2017-11-15,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30809,p1DAEZNyJ8E,2018-04-25,Avengers: Infinity War | Marvel Family Feud,Marvel Studios Movies,29,2018-04-22 23:21:45,"Avengers: Infinity War|""Avengers""|""scarlett jo...",534906,15469,331,1866,https://i.ytimg.com/vi/p1DAEZNyJ8E/default.jpg,False,False,False,NaN,2018-04-22,NaN
31014,BqD0umu1mRA,2018-04-26,Что произошло в Армении и что нужно делать нам,Алексей Навальный,29,2018-04-25 13:57:27,"Навальный|""Навальный2018""|""Фонд борьбы с корру...",1263320,132892,22193,31106,https://i.ytimg.com/vi/BqD0umu1mRA/default.jpg,False,False,False,"Когда власть врет и ворует, у людей появляется...",2018-04-25,NaN
31045,p1DAEZNyJ8E,2018-04-26,Avengers: Infinity War | Marvel Family Feud,Marvel Studios Movies,29,2018-04-22 23:21:45,"Avengers: Infinity War|""Avengers""|""scarlett jo...",810302,21878,428,2414,https://i.ytimg.com/vi/p1DAEZNyJ8E/default.jpg,False,False,False,NaN,2018-04-22,NaN
31698,OxN0qnqaYx0,2018-04-29,Avengers: Infinity War cast plays 'Friends For...,Marvel Studios Movies,29,2018-04-28 18:08:42,"Avengers: Infinity War|""Avengers""|""elizabeth o...",58116,1585,12,89,https://i.ytimg.com/vi/OxN0qnqaYx0/default.jpg,False,False,False,NaN,2018-04-28,NaN


We see that the category_id for all the missing category_name values is 29. Upon looking at the list of unique category names in the categoryids json for Canada, we see that there is an entry labeled 'NaN.' There are 74 rows with missing category_name information, and in later analyses, I will either remove these rows from my dataset or extrapolate the category of these videos by analyzing the video titles and descriptions.

In [17]:
#Reordering columns, replacing 'category_id' with 'category_name', and removing 'thumbnail_link'

df_US = df_US[['video_id', 'trending_date', 'publish_date', 'publish_time', 'channel_title', 'category_id', 'category_name', 'title', 'description', 'tags', 'views', 'likes', 'dislikes', 'comment_count', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed']]
df_CA = df_CA[['video_id', 'trending_date', 'publish_date', 'publish_time', 'channel_title', 'category_id', 'category_name', 'title', 'description', 'tags', 'views', 'likes', 'dislikes', 'comment_count', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed']]

I converted publish_time to publish_date so that I can make a better comparison between when a video is uploaded and when it trended (since we only have dates for trending_date). However, I am holding on to publish_time if I want to explore times when videos end up being uploaded on average and other such analyses.

In [18]:
print("Our Canadian data covers videos that trended at any point from {} to {}, and the videos could have been published at any point between {} and {}.".format(min(df_CA.trending_date.dt.date), max(df_CA.trending_date.dt.date), min(df_CA.publish_date.dt.date), max(df_CA.publish_date.dt.date)))
print("Our American data, on the other hand, covers videos that trended at any point from {} to {}, and the videos could have been published at any point between {} and {}.".format(min(df_US.trending_date.dt.date), max(df_US.trending_date.dt.date), min(df_US.publish_date.dt.date), max(df_US.publish_date.dt.date)))

Our Canadian data covers videos that trended at any point from 2017-11-14 to 2018-06-14, and the videos could have been published at any point between 2008-01-13 and 2018-06-14.
Our American data, on the other hand, covers videos that trended at any point from 2017-11-14 to 2018-06-14, and the videos could have been published at any point between 2006-07-23 and 2018-06-14.


In [19]:
df_CA.shape

(40881, 17)

In [20]:
df_US.shape

(40949, 17)

We see that there are 40949 rows and 17 columns in df_US, and there are 40881 rows and 17 columns in df_CA.

In [21]:
df_US.describe()

,category_id,views,likes,dislikes,comment_count
count,40949.000000,4.094900e+04,4.094900e+04,4.094900e+04,4.094900e+04
mean,19.972429,2.360785e+06,7.426670e+04,3.711401e+03,8.446804e+03
std,7.568327,7.394114e+06,2.288853e+05,2.902971e+04,3.743049e+04
min,1.000000,5.490000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,2.423290e+05,5.424000e+03,2.020000e+02,6.140000e+02
50%,24.000000,6.818610e+05,1.809100e+04,6.310000e+02,1.856000e+03
75%,25.000000,1.823157e+06,5.541700e+04,1.938000e+03,5.755000e+03
max,43.000000,2.252119e+08,5.613827e+06,1.674420e+06,1.361580e+06


In [22]:
df_CA.describe()

,category_id,views,likes,dislikes,comment_count
count,40881.000000,4.088100e+04,4.088100e+04,4.088100e+04,4.088100e+04
mean,20.795553,1.147036e+06,3.958269e+04,2.009195e+03,5.042975e+03
std,6.775054,3.390913e+06,1.326895e+05,1.900837e+04,2.157902e+04
min,1.000000,7.330000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,20.000000,1.439020e+05,2.191000e+03,9.900000e+01,4.170000e+02
50%,24.000000,3.712040e+05,8.780000e+03,3.030000e+02,1.301000e+03
75%,24.000000,9.633020e+05,2.871700e+04,9.500000e+02,3.713000e+03
max,43.000000,1.378431e+08,5.053338e+06,1.602383e+06,1.114800e+06


In [23]:
df_US.isnull().sum()

video_id                    0
trending_date               0
publish_date                0
publish_time                0
channel_title               0
category_id                 0
category_name               0
title                       0
description               570
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
comments_disabled           0
ratings_disabled            0
video_error_or_removed      0
dtype: int64

In [24]:
df_CA.isnull().sum()

video_id                     0
trending_date                0
publish_date                 0
publish_time                 0
channel_title                0
category_id                  0
category_name               74
title                        0
description               1296
tags                         0
views                        0
likes                        0
dislikes                     0
comment_count                0
comments_disabled            0
ratings_disabled             0
video_error_or_removed       0
dtype: int64

We see that the only two columns that have null entries is the description column and the category_name column. There are more missing values in the Canadian data than in the American data.

In [25]:
null_descrips = df_US.description.isnull()
null_cats = pd.Series(df_US.category_name[null_descrips])
null_cats.describe()

count                570
unique                14
top       People & Blogs
freq                 149
Name: category_name, dtype: object

These are the various categories in df_US that have videos where the descriptions are null. We see that the category which has the maximum number of videos with missing descriptions is 'People & Blogs.'

In [26]:
null_descrips_CA = df_CA.description.isnull()
null_cats_CA = pd.Series(df_CA.category_name[null_descrips])
null_cats_CA.describe()

count               569
unique               15
top       Entertainment
freq                180
Name: category_name, dtype: object

We see that the category which has the maximum number of videos with missing descriptions in df_CA is 'Entertainment.'

In [27]:
len(df_US) - len(df_US.video_id.unique())

34598

In [28]:
len(df_CA) - len(df_CA.video_id.unique())

16454

We see that there are 34598 video_ids in df_US and 16454 video_ids in df_CA that are not unique, i.e. they appear more than once in the dataframe. This could be because of a number of reasons. The video could have trended at different instances of time, say once when it was published and another when something the creator published on Twitter caused a lot of people to watch an old video of theirs. Another possible reason could be that the dataset keeps outputting a new record for every day that the video remains trending.

In [29]:
df_US[df_US.video_id == '1ZAPwfrtAFY'].sort_values(by='trending_date')

,video_id,trending_date,publish_date,publish_time,channel_title,category_id,category_name,title,description,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed
1,1ZAPwfrtAFY,2017-11-14,2017-11-13,2017-11-13 07:30:00,LastWeekTonight,24,Entertainment,The Trump Presidency: Last Week Tonight with J...,"One year after the presidential election, John...","last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,False,False,False
221,1ZAPwfrtAFY,2017-11-15,2017-11-13,2017-11-13 07:30:00,LastWeekTonight,24,Entertainment,The Trump Presidency: Last Week Tonight with J...,"One year after the presidential election, John...","last week tonight trump presidency|""last week ...",4632016,131784,9892,18284,False,False,False
454,1ZAPwfrtAFY,2017-11-16,2017-11-13,2017-11-13 07:30:00,LastWeekTonight,24,Entertainment,The Trump Presidency: Last Week Tonight with J...,"One year after the presidential election, John...","last week tonight trump presidency|""last week ...",5086221,137819,10468,18395,False,False,False
693,1ZAPwfrtAFY,2017-11-17,2017-11-13,2017-11-13 07:30:00,LastWeekTonight,24,Entertainment,The Trump Presidency: Last Week Tonight with J...,"One year after the presidential election, John...","last week tonight trump presidency|""last week ...",5392036,142006,10797,18930,False,False,False
930,1ZAPwfrtAFY,2017-11-18,2017-11-13,2017-11-13 07:30:00,LastWeekTonight,24,Entertainment,The Trump Presidency: Last Week Tonight with J...,"One year after the presidential election, John...","last week tonight trump presidency|""last week ...",5661444,145774,11064,19365,False,False,False
1158,1ZAPwfrtAFY,2017-11-19,2017-11-13,2017-11-13 07:30:00,LastWeekTonight,24,Entertainment,The Trump Presidency: Last Week Tonight with J...,"One year after the presidential election, John...","last week tonight trump presidency|""last week ...",5893503,148741,11289,19616,False,False,False
1374,1ZAPwfrtAFY,2017-11-20,2017-11-13,2017-11-13 07:30:00,LastWeekTonight,24,Entertainment,The Trump Presidency: Last Week Tonight with J...,"One year after the presidential election, John...","last week tonight trump presidency|""last week ...",6109402,151250,11508,19820,False,False,False


Upon examining one example of a video id, we see that the video was published on 2017-11-13 and trended for the first time on 2017-11-14. It continued trending till 2017-11-20 and there is a record for every day that it remained trending, with an updated number of likes, comments, views, and dislikes.

In [30]:
df_US.dtypes

video_id                          object
trending_date             datetime64[ns]
publish_date              datetime64[ns]
publish_time              datetime64[ns]
channel_title                     object
category_id                        int64
category_name                     object
title                             object
description                       object
tags                              object
views                              int64
likes                              int64
dislikes                           int64
comment_count                      int64
comments_disabled                   bool
ratings_disabled                    bool
video_error_or_removed              bool
dtype: object

In [31]:
df_CA.dtypes

video_id                          object
trending_date             datetime64[ns]
publish_date              datetime64[ns]
publish_time              datetime64[ns]
channel_title                     object
category_id                        int64
category_name                     object
title                             object
description                       object
tags                              object
views                              int64
likes                              int64
dislikes                           int64
comment_count                      int64
comments_disabled                   bool
ratings_disabled                    bool
video_error_or_removed              bool
dtype: object

Let us see how many unique channel names there are in our dataset:

In [32]:
len(df_US.channel_title.unique())

2207

In [33]:
len(df_CA.channel_title.unique())

5076

Now let us see how many unique videos there are in our dataset:

In [34]:
len(df_US.video_id.unique())

6351

In [35]:
len(df_CA.video_id.unique())

24427

In [36]:
len(df_US.title.unique())

6455

In [37]:
len(df_CA.title.unique())

24573

It is interesting that the humber of unique video IDs is not equal to the number of unique video titles. I will examine this further while performing EDA.

Let us see if uppercase titles are most commonly found among trending videos than lowercase/regularly capitalized ones:

In [38]:
n = 0
for charac in df_US.title.unique():
    if charac == charac.upper():
        n += 1
    else:
        continue
print(n)

348


We see that out of 6455 unique video titles in our dataset, 348 are in uppercase, which shows that it is in the minority.

In [39]:
df_US.sort_values("trending_date", inplace=True)

In [40]:
df_US.fillna(value='Empty', inplace=True)

For some reason, I am unable to use sort_values() or fillna() with df_CA. This needs to be further analyzed.

Let us create a dataframe where there are no duplicates, and only the first occurence of each trend is present.

In [42]:
firsts_US = df_US.drop_duplicates(subset=['video_id'])
firsts_CA = df_CA.drop_duplicates(subset=['video_id'])
firsts_CA.head()

,video_id,trending_date,publish_date,publish_time,channel_title,category_id,category_name,title,description,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed
0,n1WpP7iowLc,2017-11-14,2017-11-10,2017-11-10 17:00:03,EminemVEVO,10,Music,Eminem - Walk On Water (Audio) ft. Beyoncé,Eminem's new track Walk on Water ft. Beyoncé i...,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,False,False,False
1,0dBIkQ4Mz1M,2017-11-14,2017-11-13,2017-11-13 17:00:00,iDubbbzTV,23,Comedy,PLUSH - Bad Unboxing Fan Mail,STill got a lot of packages. Probably will las...,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,False,False,False
2,5qpjK5DgCt4,2017-11-14,2017-11-12,2017-11-12 19:05:24,Rudy Mancuso,23,Comedy,"Racist Superman | Rudy Mancuso, King Bach & Le...",WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,False,False,False
3,d380meD0W0M,2017-11-14,2017-11-12,2017-11-12 18:01:41,nigahiga,24,Entertainment,I Dare You: GOING BALD!?,I know it's been a while since we did this sho...,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,False,False,False
4,2Vv-BfVoq4g,2017-11-14,2017-11-09,2017-11-09 11:04:14,Ed Sheeran,10,Music,Ed Sheeran - Perfect (Official Music Video),🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,False,False,False


In [43]:
lasts_US = df_US.drop_duplicates(subset=['video_id'], keep='last')
lasts_CA = df_CA.drop_duplicates(subset=['video_id'], keep='last')
lasts_US.head()

,video_id,trending_date,publish_date,publish_time,channel_title,category_id,category_name,title,description,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed
127,htvR_dBs3eg,2017-11-14,2017-11-10,2017-11-10 21:38:57,theneedledrop,10,Music,Sam Smith - The Thrill of It All ALBUM REVIEW,Listen: https://www.youtube.com/watch?v=J_ub7E...,"album|""review""|""music""|""reviews""|""indie""|""unde...",98422,2926,106,798,False,False,False
136,5x1FAiIq_pQ,2017-11-14,2017-11-09,2017-11-09 15:49:21,Alicia Keys,10,Music,Alicia Keys - When You Were Gone,Find out more in The Vault: http://bit.ly/AK_A...,[none],95944,1354,181,117,False,False,False
139,vd4zwINEcLY,2017-11-14,2011-03-27,2011-03-27 04:31:25,poofables,24,Entertainment,Live in the now!,"Stop torturing yourself man, you'll never affo...","cash|""Wayne's""|""World""|""wayne""|""waynes""|""fende...",95085,909,52,193,False,False,False
140,7fm7mll2qvg,2017-11-14,2017-11-10,2017-11-10 00:00:00,SigridVEVO,10,Music,Sigrid - Strangers (Lyric Video),Listen to Strangers here: https://Sigrid.lnk.t...,"Sigrid|""Strangers""|""Island""|""Records""|""Pop""",91776,4604,46,357,False,False,False
143,q-WipZ9p0wk,2017-11-14,2017-11-09,2017-11-09 14:00:08,Brothers Green Eats,26,Howto & Style,Three meals that cost me $1.50 each,Welcome to day three of cooking for the price ...,"brothers green eats|""budget cooking""|""cooking ...",77630,1991,83,208,False,False,False


firsts is a dataframe with the first occurence of each record from df. This contains information about the first time each video trended in our time range.

lasts is a dataframe with the last occurence of each record from df. This contains information about the last time each video trended in our time range. This will help us get an indication of the final number of views, likes, dislikes, and comments that a video had.

In [47]:
len(firsts_US[firsts_US.description.str.contains('http')])

5368

We see that df_US has a large number of videos with web links included in their description! Maybe this is key to trend in the 21st Century? 

When I perform EDA, I am curious to examine what are the most common websites that are linked. Is it other social media belonging to the creator? Is it other Youtube links?

In [50]:
df_US.reset_index(inplace=True)
firsts_US.reset_index(inplace=True)
lasts_US.reset_index(inplace=True)

In [51]:
df_CA.reset_index(inplace=True)
firsts_CA.reset_index(inplace=True)
lasts_CA.reset_index(inplace=True)

I am going to create two summary tables - one for continuous and one for categorical features, in order to understand the structure of the data better.

In [53]:
cont_feats_US = pd.DataFrame()
cont_feats_CA = pd.DataFrame()

In [54]:
cont_feats_US['Features'] = ['Views', 'Likes', 'Dislikes', 'Comment_Count']
cont_feats_CA['Features'] = ['Views', 'Likes', 'Dislikes', 'Comment_Count']

In [55]:
cols = ['views', 'likes', 'dislikes', 'comment_count']

In [56]:
np.percentile(df_US['views'], 75)

1823157.0

In [57]:
cont_feats_US['Count'] = [len(df_US[col]) for col in cols]
cont_feats_CA['Count'] = [len(df_CA[col]) for col in cols]
cont_feats_US['% Missing'] = [0.0, 0.0, 0.0, 0.0]
cont_feats_CA['% Missing'] = [0.0, 0.0, 0.0, 0.0]
cont_feats_US['Cardinality'] = [len(lasts_US[col].unique()) for col in cols]
cont_feats_CA['Cardinality'] = [len(lasts_CA[col].unique()) for col in cols]
cont_feats_US['Min'] = [np.min(lasts_US[col]) for col in cols]
cont_feats_CA['Min'] = [np.min(lasts_CA[col]) for col in cols]
cont_feats_US['Q1'] = [np.percentile(lasts_US[col], 25) for col in cols]
cont_feats_CA['Q1'] = [np.percentile(lasts_CA[col], 25) for col in cols]
cont_feats_US['Mean'] = [round(np.mean(lasts_US[col]), 1) for col in cols]
cont_feats_CA['Mean'] = [round(np.mean(lasts_CA[col]), 1) for col in cols]
cont_feats_US['Median'] = [np.median(lasts_US[col]) for col in cols]
cont_feats_CA['Median'] = [np.median(lasts_CA[col]) for col in cols]
cont_feats_US['Q3'] = [np.percentile(lasts_US[col], 75) for col in cols]
cont_feats_CA['Q3'] = [np.percentile(lasts_CA[col], 75) for col in cols]
cont_feats_US['Max'] = [np.max(lasts_US[col]) for col in cols]
cont_feats_CA['Max'] = [np.max(lasts_CA[col]) for col in cols]
cont_feats_US['Std. Dev.'] = [round(np.std(lasts_US[col]), 1) for col in cols]
cont_feats_CA['Std. Dev.'] = [round(np.std(lasts_CA[col]), 1) for col in cols]

In [58]:
cont_feats_US

,Features,Count,% Missing,Cardinality,Min,Q1,Mean,Median,Q3,Max,Std. Dev.
0,Views,40949,0.0,6335,559,159156.5,1962117.2,518107.0,1473581.5,225211923,7059501.2
1,Likes,40949,0.0,5633,0,2772.5,55575.1,11906.0,38428.5,5613827,193815.5
2,Dislikes,40949,0.0,2624,0,127.0,3067.2,444.0,1507.0,1674420,31986.0
3,Comment_Count,40949,0.0,3771,0,372.0,6450.7,1266.0,4061.5,1361580,33821.0


In [59]:
cont_feats_CA

,Features,Count,% Missing,Cardinality,Min,Q1,Mean,Median,Q3,Max,Std. Dev.
0,Views,40881,0.0,24123,1023,113793.5,835571.1,287942.0,718244.5,137843120,2615592.7
1,Likes,40881,0.0,15187,0,1519.0,25821.4,5532.0,18745.0,5053338,95143.5
2,Dislikes,40881,0.0,3866,0,75.0,1277.9,223.0,677.5,1602383,12502.6
3,Comment_Count,40881,0.0,7128,0,298.0,3374.2,964.0,2592.0,1114800,14926.4


In [60]:
cat_feats_US = pd.DataFrame()
cat_feats_CA = pd.DataFrame()

In [61]:
cat_feats_US['Features'] = ['Trending_Date', 'Publish_Date', 'Channel_Title', 'Category_Name']
cat_feats_CA['Features'] = ['Trending_Date', 'Publish_Date', 'Channel_Title', 'Category_Name']

In [80]:
cols = ['trending_date', 'publish_date', 'channel_title', 'category_name']

In [81]:
cat_feats_US['% Missing'] = [0.0, 0.0, 0.0, 0.0]
cat_feats_CA['% Missing'] = [0.0, 0.0, 0.0, 0.2]
cat_feats_US['Cardinality'] = [len(lasts_US[col].unique()) for col in cols]
cat_feats_CA['Cardinality'] = [len(lasts_CA[col].unique()) for col in cols]
cat_feats_US['Mode'] = ['2018-06-14', '2018-02-05', 'ESPN', 'Entertainment']
cat_feats_CA['Mode'] = ['2018-04-07 and 2018-06-14', '2018-01-29', 'VikatanTV', 'Entertainment']
cat_feats_US['Mode Freq'] = [200, 71, 84, 1621]
cat_feats_CA['Mode Freq'] = [200, 196, 159, 8245]
cat_feats_US['Mode %'] = cat_feats_US['Mode Freq']/len(lasts_US) * 100
cat_feats_CA['Mode %'] = cat_feats_CA['Mode Freq']/len(lasts_CA) * 100
cat_feats_US['2nd Mode'] = ['2018-01-09', '2017-12-13', 'TheEllenShow', 'Music']
cat_feats_CA['2nd Mode'] = ['2018-03-14', '2018-05-13', 'Set India', 'News & Politics']
cat_feats_US['2nd Mode Freq'] = [141, 70, 74, 801]
cat_feats_CA['2nd Mode Freq'] = [192, 173, 154, 2940]
cat_feats_US['2nd Mode %'] = cat_feats_US['2nd Mode Freq']/len(lasts_US) * 100
cat_feats_CA['2nd Mode %'] = cat_feats_CA['2nd Mode Freq']/len(lasts_CA) * 100

In [82]:
cat_feats_US

,Features,% Missing,Cardinality,Mode,Mode Freq,Mode %,2nd Mode,2nd Mode Freq,2nd Mode %
0,Trending_Date,0.0,205,2018-06-14,200,3.149110,2018-01-09,141,2.220123
1,Publish_Date,0.0,313,2018-02-05,71,1.117934,2017-12-13,70,1.102189
2,Channel_Title,0.0,2198,ESPN,84,1.322626,TheEllenShow,74,1.165171
3,Category_Name,0.0,16,Entertainment,1621,25.523540,Music,801,12.612187


In [83]:
cat_feats_CA

,Features,% Missing,Cardinality,Mode,Mode Freq,Mode %,2nd Mode,2nd Mode Freq,2nd Mode %
0,Trending_Date,0.0,205,2018-04-07 and 2018-06-14,200,0.818766,2018-03-14,192,0.786015
1,Publish_Date,0.0,267,2018-01-29,196,0.802391,2018-05-13,173,0.708233
2,Channel_Title,0.0,5065,VikatanTV,159,0.650919,Set India,154,0.630450
3,Category_Name,0.2,17,Entertainment,8245,33.753633,News & Politics,2940,12.035862


Saving the dataframes:

In [85]:
file = open('US.pkl','wb')

pickle.dump(df_US, file)
pickle.dump(firsts_US, file)
pickle.dump(lasts_US, file)
pickle.dump(cont_feats_US, file)
pickle.dump(cat_feats_US, file)

file.close()

In [86]:
file = open('CA.pkl','wb')

pickle.dump(df_CA, file)
pickle.dump(firsts_CA, file)
pickle.dump(lasts_CA, file)
pickle.dump(cont_feats_CA, file)
pickle.dump(cat_feats_CA, file)

file.close()